In [23]:
import numpy as np
import pandas as pd 
#import matplotlib.pyplot as plt
#import seaborn as sns
pd.set_option('max_columns', 50)
from sklearn.linear_model import Ridge, Lasso 
from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error,explained_variance_score

from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go



In [2]:
#df = pd.ExcelFile('C:/Users/halil/Desktop/soalr_data.xlsx').parse('sheet 1')
weather_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="weather")
guneko_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="1000255-GUNEKO")

In [5]:
guneko_production = guneko_raw[["Date","Production"]]
guneko_gti = guneko_raw[["Date.1","GTI"]]


weather_guneko = weather_raw.loc[weather_raw.name ==1000255]
ali = pd.merge(guneko_production,weather_guneko,left_on="Date",right_on="date")
dataset = ali.drop(["name","date","lat","lon"],axis=1)

In [6]:
conversion = dataset["Date"].astype(str)

dataset["year"] = conversion.apply(lambda x:int(x[:4]))
dataset["month"] = conversion.apply(lambda x:int(x[5:7]))
dataset["day"] = conversion.apply(lambda x:int(x[8:10]))
dataset["hour"] = conversion.apply(lambda x:int(x[11:13]))
onehot_encoded = dataset.drop(["Date"],axis=1)

In [26]:
onehot_encoded

,Production,temperature,cloud cover,global_rad:W,diffuse_rad:W,direct_rad:W,relative_humidity_2m:p,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C,sun_elevation:d,year,month,day,hour
0,0.00000,0.348214,0.327,0.000000,0.000000,0.000000,0.942166,0.051020,0.0,0.343797,0.005333,2020,1,1,0
1,0.00000,0.330357,0.241,0.000000,0.000000,0.000000,0.974763,0.091837,0.0,0.328849,0.006667,2020,1,1,1
2,0.00000,0.316071,0.175,0.000000,0.000000,0.000000,0.985279,0.102041,0.0,0.316891,0.063333,2020,1,1,2
3,0.00000,0.308929,0.120,0.000000,0.000000,0.000000,0.985279,0.091837,0.0,0.310912,0.133333,2020,1,1,3
4,0.00000,0.282143,0.817,0.000000,0.000000,0.000000,1.000000,0.102041,0.0,0.288490,0.212000,2020,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19031,0.41041,0.398214,0.822,0.573056,0.676958,0.323005,0.372240,0.377551,0.0,0.439462,0.814000,2022,3,22,11
19032,0.00000,0.419643,0.862,0.645313,0.720318,0.385446,0.343849,0.357143,0.0,0.499253,0.849333,2022,3,22,12
19033,0.00000,0.430357,0.965,0.525984,0.756413,0.226643,0.348055,0.326531,0.0,0.485800,0.852667,2022,3,22,13
19034,0.00000,0.425000,0.992,0.361083,0.665380,0.080047,0.357518,0.244898,0.0,0.458894,0.823333,2022,3,22,14


In [7]:
def calculate_ridge(train_x,test_x,train_y,test_y):
    ridge_reg = Ridge(alpha=1) 

    ridge_reg.fit(train_x,train_y)

    ridge_train_1 = ridge_reg.predict(train_x)
    ridge_test_1 = ridge_reg.predict(test_x)

    return mean_squared_error(train_y, ridge_train_1), mean_squared_error(test_y, ridge_test_1)



def calculate_lasso(train_x_2,test_x_2,train_y_2,test_y_2):

    lasso_reg = Lasso(alpha=0.1)
    lasso_reg.fit(train_x_2,train_y_2)
    lasso_train_2 = lasso_reg.predict(train_x_2)
    lasso_test_2 = lasso_reg.predict(test_x_2)
    
    return mean_squared_error(train_y_2, lasso_train_2), mean_squared_error(test_y_2, lasso_test_2)

In [8]:

def metrics(y_train1,y_train_pred1,y_test1,y_test_pred1):
    print("for train data, mean squared error is",mean_squared_error(y_train1, y_train_pred1))
    print("for train data, mean absolute error is:",mean_absolute_error(y_train1, y_train_pred1))
    print("for train data, mean absolute percentage error is",mean_absolute_percentage_error(y_train1, y_train_pred1),"\n")

    print("for test data, mean squared error is",mean_squared_error(y_test1, y_test_pred1))
    print("for test data, mean absolute error is:",mean_absolute_error(y_test1, y_test_pred1))
    print("for test data, mean absolute percentage error is",mean_absolute_percentage_error(y_test1, y_test_pred1),"\n")


In [9]:
#normalization in range (-1,1)
def normalize(df,column_list):
    for col in column_list:
        feature_range = (0,1)
        min_max_scaler = MinMaxScaler(feature_range=feature_range)

        df[col] = min_max_scaler.fit_transform(df[col].values.reshape(-1,1))

    return df

In [10]:
def get_year(whole_data,target_year):

    whole_data = whole_data.loc[whole_data.year== target_year]
    whole_data = whole_data.drop(["year"],axis=1)
    whole_data = whole_data.reset_index(drop=True)

    return whole_data

In [11]:
def make_data(dataframe):
    train = dataframe[:len(dataframe)- 24*5]
    test = dataframe[len(dataframe)- 24*5:]

    x_train = train.copy()
    x_test = test.copy()

    y_train = x_train.pop('Production')
    y_test = x_test.pop('Production')

    return x_train, x_test, y_train, y_test 

In [12]:
def get_months(target_data):
    months = []

    for i in range(1,13):
        month_data = target_data.loc[target_data["month"]==i].copy()
        aw1 = month_data.reset_index(drop=True)
        aw1 = aw1.drop(["month"],axis=1)
        months.append(aw1)

    return months

In [13]:
normalized = normalize(onehot_encoded,onehot_encoded.columns[0:11])
data_2021 = get_year(normalized,2021)
data_months = get_months(data_2021)


In [14]:
x_train, x_test, y_train, y_test = make_data(data_months[0])

In [27]:
def calculate_errors(month_data):
    errors = []
    for i in range(0,12):
        
        x_train, x_test, y_train, y_test = make_data(month_data[i])
        ridge = calculate_ridge(x_train, x_test, y_train, y_test)

        lasso = calculate_lasso(x_train, x_test, y_train, y_test)

        errors.append({f"ridge":ridge,"lasso":lasso})
    return errors

In [29]:
errors = calculate_errors(data_months)

In [17]:
ridge_train =[]
ridge_test = []
lasso_train = []
lasso_test = []
for i in errors:
    ridge_train.append(i["ridge"][0])
    ridge_test.append(i["ridge"][1])
    lasso_train.append(i["lasso"][0])
    lasso_test.append(i["lasso"][1])

aylar = ["January","February","March","April","May","June","July","August","September","October","November","December"]

seri1  = pd.Series(ridge_train) 
seri2  = pd.Series(ridge_test) 
seri3  = pd.Series(lasso_train) 
seri4  = pd.Series(lasso_test) 
seri5 = pd.Series(aylar)

frame = {"months":seri5,"ridge_train":ridge_train, "ridge_test":ridge_test,"lasso_train":lasso_train,"lasso_test":lasso_test}
result = pd.DataFrame(frame)

In [18]:

data = result

fig = go.Figure()
fig.add_trace(go.Bar(x=data["months"],y=data["ridge_test"],name='Ridge Model Train Results', marker_color='indianred'))

fig.add_trace(go.Bar(x=data["months"], y=data["ridge_test"], name='Ridge Model Test Results',marker_color='lightsalmon'))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45,title="(-1,1) normalization, 3 encoding")
fig.show()

In [19]:

fig = go.Figure()
fig.add_trace(go.Bar(x=data["months"],y=data["lasso_test"],name='Lasso Model Train Results', marker_color='indianred'))

fig.add_trace(go.Bar(x=data["months"], y=data["lasso_test"], name='Lasso Model Test Results',marker_color='lightsalmon'))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()